# Calculate precipitation sum from GeoTiff radar images

Finnish Meteorological Institute is providing radar images as GeoTiff in S3 bucket. The data is free and openly available with CC4BY license.

This is a short and simple example how to calculate accumulated precipitation amount from GeoTiff radar images of precipitation rate (rr) with 5 minute interval. The example is ment to illustrate how to use the data with python.  

First we install necessary libraries. Rasterio (https://github.com/mapbox/rasterio) is used to fetch the data and pyproj (https://pypi.org/project/pyproj/) to calculate coordinate transformations.

In [ ]:
!pip install rasterio[s3]

In [2]:
import rasterio as rio
from datetime import datetime, timedelta
from pyproj import Proj, Transformer

Small helper function to handle time steps.

In [3]:
def roundTime(dt, roundTo=300):
   """
   Round a datetime object to any time lapse in seconds
   
   dt      : datetime.datetime object
             DateTime to round
   roundTo : int
             Closest number of seconds to round to, default 1 minute. 
   
   Returns
   -------
   datetime.datetime object
            Modified datetime   
   """
   seconds = (dt.replace(tzinfo=None) - dt.min).seconds
   rounding = (seconds+roundTo/2) // roundTo * roundTo
   return dt + timedelta(0,rounding-seconds,-dt.microsecond)

Following function opens the geotiff image from S3 bucket and sample pixel value from requested coordinates. Note that images are in EPSG:3067 projection so we need to reproject the coordinates. 

Finally, pixel values are converted to millimeters. Conversions are documented here: https://en.ilmatieteenlaitos.fi/open-data-manual-radar-data.

In [4]:
def process_timestep(bucket, key, x, y, mode='rr', timestep=300):
    """ Process time step
    
    bucket   : str 
               S3 bucket name (where radar images exist)
    key      : str
               Radar image key in the bucket
    lat      : float
               Latitude of the point of interest
    lon      : float
               Longitude of the point of interest    
    mode     : str
               Defines how pixel values are converted to millimeters
    timestep : int
               Time step of radar images (used in pixel --> mm conversion)

    Returns
    -------
    float 
         Precipitation amount in millimeters
    """    
    with rio.open('s3://{}/{}'.format(bucket, key)) as src:
        for val in src.sample([(x, y)]):
            if mode == 'rr':
                return val[0]*.01*timestep/3600
            else:
                raise Exception('mode not implemented')
        
    return 0

Last, we define a function to go through requested time range

In [5]:
def process_time_range(bucket, starttime, endtime, lat, lon, timestep, mode = 'rr', filename_start='geotiff/', filename_end='_SUOMI500-rr.tif'):
    """ Process time range 
    bucket    : str 
                S3 bucket name (where radar images exist)
    starttime : str
                Start time of the time range in format %Y-%m-%d %H:%M:%S
    endtime   : str
                End time of the time range in format %Y-%m-%d %H:%M:%S                
    lat       : float
                Latitude of the point of interest
    lon       : float
                Longitude of the point of interest    
    timestep  : int
                Time step of radar images
    mode      : str
                Defines how pixel values are converted to millimeters
    filename_start : str
                     For example path of the image. Images are assumed to found with name 
                     filename_start+timestamp+filename_end.  
    filename_start : str
                 Images are assumed to found with name filename_start+timestamp+filename_end.  
                      
    Returns
    -------
    float 
         Precipitation amount in millimeters
    
    """
    
    start = roundTime(datetime.strptime(starttime, '%Y-%m-%d %H:%M:%S'))
    end = roundTime(datetime.strptime(endtime, '%Y-%m-%d %H:%M:%S'))    
    time_it = start    
    
    transformer = Transformer.from_crs("epsg:4326", "epsg:3067")
    x,y = transformer.transform(lat, lon)
    
    prec_sum = 0
    while time_it <= end:
        key = '{}{}{}'.format(filename_start, time_it.strftime('%Y%m%d%H%M'), filename_end)
        prec_sum += process_timestep(bucket, key, x, y, timestep=timestep)

        if time_it.minute == 0: print('{:.2f}'.format(prec_sum), end='')
        else: print('.', end='')
            
        time_it +=  timedelta(seconds=timestep)

    print('')
    return prec_sum

And finally we run everything:

In [6]:
bucket = 'fmi-radar-opendata-sandbox'
starttime = '2020-07-11 00:00:00'
endtime = '2020-07-11 23:59:00'
lat = 63.1593
lon = 29.8346

In [7]:
prec_sum = process_time_range(bucket, starttime, endtime, lat, lon, 300)
print('Precipitation sum at {},{} (Koli, Finland) on 11th July 2020 is {:.2f} mm'.format(lat,lon,prec_sum))

0.00...........0.00...........0.00...........0.00...........0.00...........0.00...........0.00...........0.00...........0.00...........0.00...........0.00...........0.02...........0.61...........3.26...........9.86...........11.49...........12.91...........17.49...........19.29...........19.86...........20.08...........20.08...........20.08...........20.08...........20.08
Precipitation sum at 63.1593,29.8346 (Koli, Finland) on 11th July 2020 is 20.08 mm


## Dask version

As the above computation contains almost 300 geotiff images to process, it does take some time. If we want to calculate precipitation sum over longer period or do some other heavy computation, it may be good idea to distritbute the calculation. Following example shows how this can achieved using Dask (https://docs.dask.org/en/latest/). 

First we need to install the envinronment. This example does not cover installing dask but all assets used to setup the dask cluster are included in the repository. A good tutorial for setting up the cluster can be found at: https://towardsdatascience.com/serverless-distributed-data-pre-processing-using-dask-amazon-ecs-and-python-part-1-a6108c728cc4

To use the cluster, we need several libraries to be installed. Following commands install the require libraries. Versions of each libraries are defined as they need to mach with the cluster.

In [ ]:
!pip uninstall -y enum34
!pip install blosc==1.8.3
!pip install --upgrade joblib==0.15.1 dask[dataframe]==2.12.0 distributed==2.12.0 lz4==3.0.2 cloudpickle==1.3.0 msgpack==1.0.0 numpy==1.18.1 toolz==0.10.0 tornado==6.0.4

Next we import necessary libraries and initialize dask client

In [9]:
from dask.distributed import Client
from dask import delayed
import dask

In [ ]:
client = Client('Dask-Scheduler.local-dask:8786')
client

Now we can define necessary variables. These are the same as in previous example except that we have a longer time range and we use start and end times as DateTime object.

In [11]:
bucket = 'fmi-radar-opendata-sandbox'
lat = 63.1593
lon = 29.8346
timestep=300
filename_start='geotiff/'
filename_end='_SUOMI500-rr.tif'
starttime = datetime.strptime('2020-07-11 00:00:00', "%Y-%m-%d %H:%M:%S")
endtime = datetime.strptime('2020-07-15 23:59:00', "%Y-%m-%d %H:%M:%S")

Computing is also almost similar to local version. We go through the time steps but instead of directly calculating the value, we define the calculation and call dask to compute all at once. Dask handles the distribution and gathering results.

In [12]:
transformer = Transformer.from_crs("epsg:4326", "epsg:3067")
x,y = transformer.transform(lat, lon)

dfs, df = [], []
time_it = starttime
while time_it <= endtime:
    key = '{}{}{}'.format(filename_start, time_it.strftime('%Y%m%d%H%M'), filename_end)
    dfs.append(delayed(process_timestep)(bucket, key, x, y, timestep=timestep))
    time_it +=  timedelta(seconds=timestep)

df = dask.compute(*dfs)

print('Precipitation sum at {},{} (Koli, Finland) on between {} - {} is {:.2f} mm'.format(lat,
                                                                                          lon,
                                                                                          starttime.strftime('%dth %b %Y'), 
                                                                                          endtime.strftime('%dth %b %Y'), 
                                                                                          sum(df)))

Precipitation sum at 63.1593,29.8346 (Koli, Finland) on between 11th Jul 2020 - 15th Jul 2020 is 28.77 mm
